In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd.profiler import record_function

In [2]:
import sys
sys.path.insert(1, "../../../")

In [3]:
from data_preprocessing import get_means, get_stds, ImageNetSubset, get_norms
from Models.yolov8cls_path import Model
from evaluation import count_parameters, show_tree, generate_dataframe

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 
device=torch.device('cpu') 

In [8]:
data_path = '../../../../datasets/imagenette2/'
norms_path = os.path.join(data_path, 'norms.json')
norms = get_norms(path=norms_path, train_loader=None)
means, stds = norms['means'], norms['stds']

# Inference at 640x640

In [9]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((640, 640)),
                                      transforms.Normalize(mean=means, std=stds)])

In [10]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [11]:
from torch.profiler import profile, ProfilerActivity

In [12]:
model = Model(num_classes=1000, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=True,
              bottleneck=0.75, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [12]:
model.eval()
model.training

False

In [13]:
model

Model(
  (conv1): Conv(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (conv2): Conv(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (c2f1): C2f(
    (conv1): Conv(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (conv2): Conv(
      (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (n_blocks): ModuleList(
      (0): Bottleneck(
        (conv1): Conv(
          (conv): Conv2d(32, 24, kernel_s

In [13]:
next(model.parameters()).device

device(type='cpu')

In [15]:
img = train_dataset[6][0].unsqueeze(0).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-6/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(12):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total MFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.06%     718.800us       100.00%        1.268s     126.836ms

In [16]:
img = train_dataset[6][0].unsqueeze(0).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-6/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(3):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total KFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.07%      69.500us       100.00%      94.148ms      94.148ms

In [17]:
df = generate_dataframe(prof)

C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Backbone experiments\YOLOv8s-cls Path\Version 6\../../..\evaluation.py:34: FutureWarning: `cuda_time` is deprecated, please use `device_time` instead.
  row[attr] = getattr(obj, attr)
C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Backbone experiments\YOLOv8s-cls Path\Version 6\../../..\evaluation.py:34: FutureWarning: `cuda_time_total` is deprecated. Use `device_time_total` instead.
  row[attr] = getattr(obj, attr)
C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Backbone experiments\YOLOv8s-cls Path\Version 6\../../..\evaluation.py:34: FutureWarning: `self_cuda_memory_usage` is deprecated. Use `self_device_memory_usage` instead.
  row[attr] = getattr(obj, attr)
C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Backbone experiments\YOLOv8s-cls Path\Version 6\../../..\evaluation.py:34: FutureWarning: `self_cuda_time_total` is deprecated. Use `self_device_time_total` instead.
  row[attr] = getattr(obj, attr)


# GFLOPs at 640x640 using Ultralytics functions

In [18]:
from ultralytics_flops import get_flops, get_flops_with_torch_profiler

In [19]:
get_flops(model, imgsz=640)

12.1899008

In [20]:
get_flops_with_torch_profiler(model, imgsz=640)

11.2878592

# GFLOPs at 224x224 using Ultralytics functions

In [21]:
from ultralytics_flops import get_flops, get_flops_with_torch_profiler

In [22]:
get_flops(model, imgsz=224)

1.493262848

In [23]:
get_flops_with_torch_profiler(model, imgsz=224)

1.382762752

# Inference at 224x224

In [24]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [25]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [26]:
from torch.profiler import profile, ProfilerActivity

In [27]:
model = Model(num_classes=1000, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=True,
              bottleneck=0.75, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [28]:
img = train_dataset[6][0].unsqueeze(0).to(device)
img.shape

torch.Size([1, 3, 224, 224])

In [29]:
model.eval()
model.training

False

In [30]:
next(model.parameters()).device

device(type='cpu')

In [31]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-6/inference_at_224'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(12):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total KFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.15%     323.800us       100.00%     217.710ms      21.771ms           0 b           0 b            10                                                                                []            --  
                       infer

In [32]:
img.shape, img.device

(torch.Size([1, 3, 224, 224]), device(type='cpu'))

In [33]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-6/inference_at_224'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(3):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total KFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.14%      39.800us       100.00%      28.187ms      28.187ms           0 b           0 b             1                                                                                []            --  
                       infer

# Count Parameters

My implementation:

In [34]:
count_parameters(model)

5877560

Ultralytics implementation:

In [35]:
model = torch.load('../../../Models/yolov8s-cls.pt')['model'].model

C:\conda_tmp\ipykernel_15856\3087971228.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('../../../Models/yolov8s-cls.pt')['model'].model


FileNotFoundError: [Errno 2] No such file or directory: '../../../Models/yolov8s-cls.pt'

In [ ]:
count_parameters(model)